# Setup

## Imports

In [16]:
import QuantumOptics as qo
using Logging

using LinearAlgebra
using SparseArrays
import CairoMakie as cm

using ProtoStructs

import QuantumOptics.⊗
import QuantumOptics.*

import CSV
using JSON
import Tables

using YAXArrays

using MiniLoggers
using Dates
using Revise

In [17]:
import SuperconductingCavities as SC

## Some Extra Stuff

In [18]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(ProgressLogger)

MiniLogger{MiniLoggers.Squash, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}, DateFormat{Symbol("yyyy-mm-dd HH:MM:SS"), Tuple{Dates.DatePart{'y'}, Dates.Delim{Char, 1}, Dates.DatePart{'m'}, Dates.Delim{Char, 1}, Dates.DatePart{'d'}, Dates.Delim{Char, 1}, Dates.DatePart{'H'}, Dates.Delim{Char, 1}, Dates.DatePart{'M'}, Dates.Delim{Char, 1}, Dates.DatePart{'S'}}}, typeof(string)}(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(19) open, 0 bytes waiting)), VSCodeServer.io_send_callback), VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Error, LogLevel(-1), Dict{Any, Int64}(), true, MiniLoggers.Token[MiniLoggers.Token("[", MiniLoggers

In [19]:
function tostr(obj)
    io = IOBuffer()
    show(io, "text/plain", obj)
    String(take!(io))
end

tostr (generic function with 1 method)

# Example

## Initializing Models

### Creating Model From Scratch

#### One Mode

In [11]:
Eᶜ = 0.10283303447280807;
Eʲ = 26.96976142643705;
Eᵒˢᶜ = 6.2280871213;
g = 0.026184375000000013;
name = "transmon";
ng  =  0;
N=10;
N_cut = 60;

OneMode = SC.Transmon_Resonators_Constructor(Eᶜ, Eʲ, [Eᵒˢᶜ], [g], N, [N], Cavity_Names = ["Mode 3"]);

#### Two Mode

In [12]:
Eᶜ = 0.10283303447280807;
Eʲ = 26.96976142643705;
Eᵒˢᶜs = [6.2280871213, 6.72062927579];
gs = [0.026184375000000013, 0.03020625000000001];
name = "transmon";
ng  =  0;
Nₜ =10;
Nᵣs = [6, 6]
N_cut = 60;


TwoModes = SC.Transmon_Resonators_Constructor(Eᶜ, Eʲ, Eᵒˢᶜs, gs, Nₜ, Nᵣs, Cavity_Names = ["Mode 3", "Mode 5"]);

### Importing Model

In [3]:
Mode3 = SC.Transmon_Resonators_Loader("ModelSaves/Mode_3/Mode_3.json");

In [15]:
Mode3.Stuff["op_drive_params"]["sb_f_0_4pi_f_3"]["Envelope Args"]

Dict{String, Any} with 3 entries:
  "pulse_time"   => 1601.37
  "sigma_factor" => 2
  "ramp_time"    => 10

In [10]:
Mode3.Stuff["Drive Sequences"]["Binomial_Code"]

18-element Vector{Any}:
 "q_e_0"
 "q_g_0"
 "sb_f_0_Main"
 "q_g_0"
 "q_e_1"
 "q_g_1"
 "half_sb_f_1"
 "q_g_1"
 "q_e_1"
 "sb_f_2_Main"
 "q_g_2"
 "sb_f_0_Main"
 "q_e_0"
 "sb_f_0_4pi_f_3"
 "q_g_1"
 "sb_f_3_Main"
 "q_e_0"
 "sb_f_1_2pi_f_3"

## Time Evolving Wavefunctions

### Running A Single Pulse

In [21]:
ψ = Mode3.dressed_states[(0,0)]
SC.RunSingleOperator(Mode3, ψ, Mode3.Stuff["op_drive_params"]["q_g_0"], save_step = true, to_return = "Nothing", step_name = "q_g_0", run_name = "Single_Op_Demo_"*string(now()));

[2024-09-01 18:02:32] Info: Running Time Evolution
[2024-09-01 18:02:32] LogLevel(-1): ODE progress = 0
[2024-09-01 18:02:32] LogLevel(-1): ODE message = dt=0.0010432581424918218	t=1.0419916278610004	max u=-0.4512538658823686 - 0.8923725857046029im, progress = 0.010399118042524955
[2024-09-01 18:02:33] LogLevel(-1): ODE message = dt=0.0010432591097843593	t=2.0852500959849563	max u=-0.7382805949277542 + 0.674420209921304im, progress = 0.02081087920144667
[2024-09-01 18:02:34] LogLevel(-1): ODE message = dt=0.001043258764323398	t=3.128509125898927	max u=0.8479033027523399 + 0.5299575367499055im, progress = 0.03122264596705516
[2024-09-01 18:02:35] LogLevel(-1): ODE message = dt=0.0010432606298179875	t=4.171768857930078	max u=0.2827994955471448 - 0.9589923961793032im, progress = 0.04163441973982114
[2024-09-01 18:02:35] LogLevel(-1): ODE message = dt=0.001043261182558991	t=5.215029448988226	max u=-0.9995964417054138 - 0.015046979091596932im, progress = 0.052046202085710834
[2024-09-01 18:

### Running a Series of Operators

In [12]:
ψ = Mode3.dressed_states[(0,0)]
SC.RunPulseSequence(Mode3, ψ, ["q_g_0", "q_e_0"])

UndefVarError: UndefVarError: `model` not defined

## With Density Matrices

In [63]:
Mode3.CandD_Ops

Dict{Any, Any} with 3 entries:
  "Mode 3 Collapse"    => Operator(dim=100x100)…
  "Transmon Collapse"  => Operator(dim=100x100)…
  "Transmon Dephasing" => Operator(dim=100x100)…

In [11]:
ψ = Mode3.dressed_states[(0,0)]
ρ = ψ*ψ'
SC.RunPulseSequence(Mode3, ρ, Mode3.Stuff["Drive Sequences"]["Binomial_Code"], c_ops = [], run_name = "Demo_Run_With_Out_Loss_"*string(now()))

[2024-09-02 10:22:16] Info: Running operator q_e_0
[2024-09-02 10:22:16] Info: Running Time Evolution
[2024-09-02 10:22:20] LogLevel(-1): ODE progress = 0
[2024-09-02 10:22:23] LogLevel(-1): ODE message = dt=0.0022297746318701603	t=2.5613361773140353	max u=0.9999013482021929 + 8.228810546230448e-15im, progress = 0.035772851638464186
[2024-09-02 10:22:25] LogLevel(-1): ODE message = dt=0.0020833495695194502	t=4.707294672719735	max u=0.9998000200400847 + 1.7106456496963785e-14im, progress = 0.06574433900446558
[2024-09-02 10:22:28] LogLevel(-1): ODE message = dt=0.0020097059359571743	t=6.750558324570673	max u=0.9998388279127534 + 2.0327635739251425e-14im, progress = 0.09428154084595913
[2024-09-02 10:22:30] LogLevel(-1): ODE message = dt=0.00196296271793647	t=8.73535858483198	max u=0.9999250840635244 + 2.672902978483838e-14im, progress = 0.12200221487195503
[2024-09-02 10:22:32] LogLevel(-1): ODE message = dt=0.0019299547416470359	t=10.680941898272785	max u=0.999849307039278 + 3.05097072

InterruptException: InterruptException:

In [ ]:
SC.RunPulseSequence(Mode3, ρ, Mode3.Stuff["Drive Sequences"]["Binomial_Code"], c_ops = collect(values(Mode3.CandD_Ops)), run_name = "Demo_Run_With_Loss_"*string(now()))